# DataHowLab SDK v2.0 - Comprehensive Examples

This notebook provides comprehensive examples of all functions available in the DataHowLab SDK v2.0.

## Table of Contents
1. [Client Initialization](#1-client-initialization)
2. [Product Operations](#2-product-operations)
3. [Variable Operations](#3-variable-operations)
4. [Experiment Operations](#4-experiment-operations)
5. [Recipe Operations](#5-recipe-operations)
6. [Project Operations](#6-project-operations)
7. [Model Operations](#7-model-operations)
8. [Dataset Operations](#8-dataset-operations)
9. [Prediction Operations](#9-prediction-operations)
10. [Error Handling](#10-error-handling)

## 1. Client Initialization

The `DataHowLabClient` is the main entry point for all SDK operations.

In [ ]:
from dhl_sdk import DataHowLabClient

# Option 1: Provide API key directly
client = DataHowLabClient(
    api_key="your-api-key",
    base_url="https://yourdomain.datahowlab.ch/"
)

# Option 2: Use environment variable DHL_API_KEY
# client = DataHowLabClient(base_url="https://yourdomain.datahowlab.ch/")

# Option 3: Disable SSL verification (for self-signed certificates)
# client = DataHowLabClient(
#     api_key="your-api-key",
#     base_url="https://yourdomain.datahowlab.ch/",
#     verify_ssl=False
# )

# Option 4: Custom timeout
# client = DataHowLabClient(
#     api_key="your-api-key",
#     base_url="https://yourdomain.datahowlab.ch/",
#     timeout=60.0  # 60 seconds
# )

print("✓ Client initialized successfully")

## 2. Product Operations

Products represent the main entity in your DataHowLab system (e.g., cell lines, processes).

### 2.1 Create Product

In [ ]:
# Create a new product
product = client.create_product(
    code="CHO001",
    name="CHO Cell Line A",
    description="Standard CHO cell line for antibody production",
    process_format="mammalian"  # or "microbial"
)

print(f"Created product: {product.name}")
print(f"  ID: {product.id}")
print(f"  Code: {product.code}")
print(f"  Process Format: {product.process_format}")

### 2.2 List Products

In [ ]:
# List all products (first 100)
products = client.list_products()
print(f"Found {len(products)} products")
for p in products[:5]:  # Show first 5
    print(f"  - {p.code}: {p.name}")

# Filter by code
products = client.list_products(code="CHO001")
print(f"\nProducts with code 'CHO001': {len(products)}")

# Get all products using pagination
all_products = client.list_products(iter_all=True)
count = 0
for product in all_products:
    count += 1
    if count >= 10:  # Limit for demo
        break
print(f"\nIterated through {count} products")

### 2.3 Get Product by ID

In [ ]:
# Get a specific product by ID
if products:
    product_id = products[0].id
    product = client.get_product(product_id)
    print(f"Retrieved product: {product.name} ({product.code})")

## 3. Variable Operations

Variables represent process parameters, measurements, and other data points.

### 3.1 Create Numeric Variable

In [ ]:
from dhl_sdk import NumericVariable

# Create a numeric variable with constraints
temperature = client.create_variable(
    code="TEMP",
    name="Temperature",
    unit="°C",
    group="X Variables",
    type=NumericVariable(
        min=20.0,
        max=40.0,
        default=37.0,
        interpolation="linear"
    ),
    description="Bioreactor temperature"
)

print(f"Created variable: {temperature.name} ({temperature.code})")
print(f"  Unit: {temperature.unit}")
print(f"  Group: {temperature.group}")
print(f"  Type: {temperature.type.kind}")
print(f"  Min: {temperature.type.min}, Max: {temperature.type.max}")

### 3.2 Create Categorical Variable

In [ ]:
from dhl_sdk import CategoricalVariable

# Create a categorical variable
phase = client.create_variable(
    code="PHASE",
    name="Process Phase",
    unit="n",
    group="Z Variables",
    type=CategoricalVariable(
        categories=["Init", "Growth", "Production", "Harvest"],
        strict=True,
        default="Init"
    ),
    description="Current phase of the process"
)

print(f"Created variable: {phase.name}")
print(f"  Categories: {phase.type.categories}")
print(f"  Strict: {phase.type.strict}")

### 3.3 Create Logical Variable

In [ ]:
from dhl_sdk import LogicalVariable

# Create a logical (boolean) variable
alarm = client.create_variable(
    code="ALARM",
    name="Alarm Status",
    unit="boolean",
    group="Z Variables",
    type=LogicalVariable(default=False),
    description="Alarm triggered status"
)

print(f"Created variable: {alarm.name}")
print(f"  Default: {alarm.type.default}")

### 3.4 Create Flow Variable

In [ ]:
from dhl_sdk import FlowVariable, FlowReference

# First, create or get the required variables
# 1. X variable (measurement)
glucose = client.create_variable(
    code="GLC",
    name="Glucose",
    unit="g/L",
    group="X Variables",
    type=NumericVariable(min=0, max=100),
    description="Glucose concentration"
)

# 2. Feed concentration variable
glc_feed_conc = client.create_variable(
    code="GLC_FEED",
    name="Glucose Feed Concentration",
    unit="g/L",
    group="Feed Concentrations",
    type=NumericVariable(min=0, max=500),
    description="Glucose concentration in feed"
)

# 3. Volume variable (get existing or create new)
volume = client.create_variable(
    code="VOL",
    name="Volume",
    unit="L",
    group="X Variables",
    type=NumericVariable(min=0, max=1000),
    description="Bioreactor volume"
)

# Now create the flow variable
feed = client.create_variable(
    code="GLC_BOLUS",
    name="Glucose Bolus Feed",
    unit="L",
    group="Feeds/Flows",
    type=FlowVariable(
        flow_type="bolus",
        references=[
            FlowReference(
                measurement=glucose.id,
                concentration=glc_feed_conc.id
            )
        ],
        volume_variable=volume.id
    ),
    description="Glucose bolus feed"
)

print(f"Created flow variable: {feed.name}")
print(f"  Flow type: {feed.type.flow_type}")
print(f"  References: {len(feed.type.references)}")

### 3.5 Create Spectrum Variable

In [ ]:
from dhl_sdk import SpectrumVariable, SpectrumAxis

# Create a spectrum variable for spectroscopy data
spectrum = client.create_variable(
    code="SPEC1",
    name="NIR Spectrum",
    unit="absorbance",
    group="Spectra",
    type=SpectrumVariable(
        x_axis=SpectrumAxis(
            dimension=1000,
            unit="nm",
            min=800,
            max=2500,
            label="Wavelength"
        ),
        y_axis=SpectrumAxis(
            label="Absorbance"
        )
    ),
    description="Near-infrared spectrum"
)

print(f"Created spectrum variable: {spectrum.name}")
print(f"  X-axis: {spectrum.type.x_axis.label} ({spectrum.type.x_axis.unit})")
print(f"  Dimension: {spectrum.type.x_axis.dimension}")

### 3.6 List Variables

In [ ]:
# List all variables
variables = client.list_variables()
print(f"Found {len(variables)} variables")

# Filter by code
temp_vars = client.list_variables(code="TEMP")
print(f"\nVariables with code 'TEMP': {len(temp_vars)}")

# Filter by group
x_vars = client.list_variables(group="X Variables")
print(f"X Variables: {len(x_vars)}")
for v in x_vars[:3]:
    print(f"  - {v.code}: {v.name}")

# Filter by type
numeric_vars = client.list_variables(variable_type="numeric")
print(f"\nNumeric variables: {len(numeric_vars)}")

categorical_vars = client.list_variables(variable_type="categorical")
print(f"Categorical variables: {len(categorical_vars)}")

## 4. Experiment Operations

Experiments store time-series data for your process runs or samples.

### 4.1 Create Run Experiment with Time-Series Data

In [ ]:
from dhl_sdk import ExperimentData, TimeseriesData
from datetime import datetime

# Define experiment time range
start_time = datetime(2025, 1, 1, 8, 0, 0)
end_time = datetime(2025, 1, 15, 16, 0, 0)

# Unix timestamps for data points
timestamps = [
    1735718400,  # Day 1
    1735804800,  # Day 2
    1735891200,  # Day 3
    1736064000,  # Day 5
    1736236800,  # Day 7
]

# Create experiment
experiment = client.create_experiment(
    name="Batch Run 2025-001",
    description="Standard production run",
    product=product,
    variables=[temperature, phase, glucose],
    data=ExperimentData(
        variant="run",
        start_time=start_time,
        end_time=end_time,
        timeseries={
            "TEMP": TimeseriesData(
                timestamps=timestamps,
                values=[37.0, 37.1, 37.0, 36.9, 37.1]
            ),
            "PHASE": TimeseriesData(
                timestamps=[timestamps[0], timestamps[2], timestamps[4]],
                values=["Init", "Growth", "Production"]
            ),
            "GLC": TimeseriesData(
                timestamps=timestamps,
                values=[5.0, 4.2, 3.8, 2.5, 1.2]
            ),
        }
    ),
    process_format="mammalian"
)

print(f"Created experiment: {experiment.name}")
print(f"  ID: {experiment.id}")
print(f"  Variables: {len(experiment.variables)}")

### 4.2 Create Spectra Experiment

In [ ]:
from dhl_sdk import SpectraExperimentData, SpectraData

# Create experiment with spectroscopy data
spectra_exp = client.create_experiment(
    name="NIR Spectra Collection 001",
    description="NIR spectra from batch run",
    product=product,
    variables=[spectrum],
    data=SpectraExperimentData(
        variant="run",
        start_time=start_time,
        end_time=end_time,
        spectra=SpectraData(
            timestamps=[1735718400, 1735804800, 1735891200],
            values=[
                [0.1, 0.2, 0.3, 0.4, 0.5],  # First spectrum
                [0.15, 0.22, 0.31, 0.39, 0.48],  # Second spectrum
                [0.12, 0.21, 0.32, 0.41, 0.51],  # Third spectrum
            ]
        ),
        inputs={
            "TEMP": [37.0, 37.1, 37.0],
            "GLC": [5.0, 4.2, 3.8]
        }
    )
)

print(f"Created spectra experiment: {spectra_exp.name}")
print(f"  ID: {spectra_exp.id}")

### 4.3 List Experiments

In [ ]:
# List all experiments
experiments = client.list_experiments()
print(f"Found {len(experiments)} experiments")
for exp in experiments[:5]:
    print(f"  - {exp.name}")

# Filter by name
batch_exps = client.list_experiments(name="Batch")
print(f"\nExperiments with 'Batch' in name: {len(batch_exps)}")

# Filter by product
product_exps = client.list_experiments(product=product)
print(f"Experiments for product {product.code}: {len(product_exps)}")

### 4.4 Get Experiment by ID

In [ ]:
# Get specific experiment
if experiments:
    exp_id = experiments[0].id
    exp = client.get_experiment(exp_id)
    print(f"Retrieved experiment: {exp.name}")

### 4.5 Get Experiment Data

In [ ]:
# Retrieve experiment data
exp_data = client.get_experiment_data(experiment)

print("Experiment data:")
for var_code, data in exp_data.items():
    num_points = len(data.get('timestamps', []))
    print(f"  {var_code}: {num_points} data points")
    if num_points > 0:
        print(f"    First timestamp: {data['timestamps'][0]}")
        print(f"    First value: {data['values'][0]}")

# Access specific variable data
if 'TEMP' in exp_data:
    temp_data = exp_data['TEMP']
    print(f"\nTemperature data:")
    print(f"  Timestamps: {temp_data['timestamps']}")
    print(f"  Values: {temp_data['values']}")

## 5. Recipe Operations

Recipes define setpoint profiles for process variables.

### 5.1 Create Recipe

In [ ]:
# Create a recipe with setpoints
recipe = client.create_recipe(
    name="Standard CHO Recipe",
    description="Standard temperature profile for CHO cells",
    product=product,
    variables=[temperature],
    duration=1209600,  # 14 days in seconds
    data={
        "TEMP": TimeseriesData(
            timestamps=[0, 259200, 604800, 1209600],  # Day 0, 3, 7, 14
            values=[36.0, 37.0, 37.0, 36.5]
        )
    }
)

print(f"Created recipe: {recipe.name}")
print(f"  ID: {recipe.id}")
print(f"  Duration: {recipe.duration} seconds")
print(f"  Variables: {len(recipe.variables)}")

### 5.2 List Recipes

In [ ]:
# List all recipes
recipes = client.list_recipes()
print(f"Found {len(recipes)} recipes")
for r in recipes[:5]:
    print(f"  - {r.name}")

# Filter by name
cho_recipes = client.list_recipes(name="CHO")
print(f"\nRecipes with 'CHO' in name: {len(cho_recipes)}")

# Filter by product
product_recipes = client.list_recipes(product=product)
print(f"Recipes for product {product.code}: {len(product_recipes)}")

## 6. Project Operations

Projects organize datasets and models for analysis.

### 6.1 List Cultivation Projects

In [ ]:
# List cultivation projects
cultivation_projects = client.list_projects(
    project_type="cultivation",
    process_format="mammalian"
)

print(f"Found {len(cultivation_projects)} cultivation projects")
for proj in cultivation_projects[:5]:
    print(f"  - {proj.name}")
    print(f"    ID: {proj.id}")
    print(f"    Type: {proj.project_type}")

### 6.2 List Spectroscopy Projects

In [ ]:
# List spectroscopy projects
spectro_projects = client.list_projects(
    project_type="spectroscopy"
)

print(f"Found {len(spectro_projects)} spectroscopy projects")
for proj in spectro_projects[:5]:
    print(f"  - {proj.name}")

### 6.3 Get Project by ID

In [ ]:
# Get specific project
if cultivation_projects:
    project_id = cultivation_projects[0].id
    project = client.get_project(project_id)
    print(f"Retrieved project: {project.name}")
    print(f"  Type: {project.project_type}")
    print(f"  Description: {project.description}")

## 7. Model Operations

Models are trained on datasets and used for predictions.

### 7.1 List Models for a Project

In [ ]:
# List all models in a project
if cultivation_projects:
    project = cultivation_projects[0]
    
    # List all models
    models = client.list_models(project)
    print(f"Found {len(models)} models in project '{project.name}'")
    
    for model in models[:5]:
        print(f"  - {model.name}")
        print(f"    Status: {model.status}")
        print(f"    Type: {model.model_type}")
        print(f"    Ready: {model.is_ready}")
    
    # Filter by model type
    prop_models = client.list_models(project, model_type="propagation")
    print(f"\nPropagation models: {len(prop_models)}")
    
    hist_models = client.list_models(project, model_type="historical")
    print(f"Historical models: {len(hist_models)}")
    
    # Filter by name
    named_models = client.list_models(project, name="MyModel")
    print(f"Models named 'MyModel': {len(named_models)}")

### 7.2 Get Model by ID

In [ ]:
# Get specific model
if models:
    model_id = models[0].id
    model = client.get_model(model_id)
    print(f"Retrieved model: {model.name}")
    print(f"  Status: {model.status}")
    print(f"  Type: {model.model_type}")
    print(f"  Dataset: {model.dataset.name if model.dataset else 'None'}")

### 7.3 Access Model Variables

In [ ]:
# Get input and output variables for a model
if models and models[0].is_ready:
    model = models[0]
    
    input_vars = model.input_variables
    output_vars = model.output_variables
    
    print(f"Model '{model.name}' variables:")
    print(f"\nInput variables ({len(input_vars)}):")
    for var in input_vars:
        print(f"  - {var.code}: {var.name}")
    
    print(f"\nOutput variables ({len(output_vars)}):")
    for var in output_vars:
        print(f"  - {var.code}: {var.name}")

## 8. Dataset Operations

Datasets group experiments for model training.

### 8.1 List Datasets for a Project

In [ ]:
# List datasets in a project
if cultivation_projects:
    project = cultivation_projects[0]
    
    datasets = client.list_datasets(project)
    print(f"Found {len(datasets)} datasets in project '{project.name}'")
    
    for ds in datasets[:5]:
        print(f"  - {ds.name}")
        print(f"    ID: {ds.id}")
        print(f"    Variables: {len(ds.variables)}")
    
    # Filter by name
    filtered_datasets = client.list_datasets(project, name="Training")
    print(f"\nDatasets with 'Training' in name: {len(filtered_datasets)}")

### 8.2 Get Datasets via Project

In [ ]:
# Alternative: Use project's get_datasets method
if cultivation_projects:
    project = cultivation_projects[0]
    datasets = project.get_datasets(client)
    print(f"Datasets from project method: {len(datasets)}")

## 9. Prediction Operations

Use trained models to make predictions on new data.

### 9.1 Spectroscopy Model Predictions

In [ ]:
from dhl_sdk import SpectraPredictionInput

# Find a spectroscopy model
if spectro_projects:
    spectro_project = spectro_projects[0]
    spectro_models = client.list_models(spectro_project)
    
    if spectro_models and spectro_models[0].is_ready:
        model = spectro_models[0]
        
        # Make predictions with spectra data
        predictions = model.predict(
            client,
            SpectraPredictionInput(
                spectra=[
                    [1.0, 2.0, 3.0, 4.0, 5.0],  # First spectrum
                    [1.1, 2.1, 3.1, 4.1, 5.1],  # Second spectrum
                    [0.9, 1.9, 2.9, 3.9, 4.9],  # Third spectrum
                ],
                inputs={
                    "TEMP": [37.0, 37.1, 37.0],
                    "GLC": [5.0, 4.5, 4.0]
                }
            )
        )
        
        print(f"Spectroscopy predictions:")
        for output_name, values in predictions.outputs.items():
            print(f"  {output_name}: {len(values)} predictions")
            print(f"    Values: {values}")
        
        if predictions.confidence_intervals:
            print(f"  Confidence intervals available")
        
        print(f"\nMetadata:")
        for key, value in predictions.metadata.items():
            print(f"  {key}: {value}")

### 9.2 Cultivation Propagation Model Predictions

In [ ]:
from dhl_sdk import PropagationPredictionInput

# Find a propagation model
if cultivation_projects:
    project = cultivation_projects[0]
    prop_models = client.list_models(project, model_type="propagation")
    
    if prop_models and prop_models[0].is_ready:
        model = prop_models[0]
        
        # Make propagation predictions
        predictions = model.predict(
            client,
            PropagationPredictionInput(
                timestamps=[0, 24, 48, 72, 96, 120, 144, 168],  # Hours
                unit="h",  # Hours
                inputs={
                    # X variables: initial values only
                    "TEMP": [37.0],
                    "GLC": [5.0],
                    
                    # W variables and flows: values for each timestamp
                    "FEED": [0.0, 0.1, 0.1, 0.2, 0.2, 0.3, 0.3, 0.4],
                },
                confidence=0.8,  # 80% confidence interval
                starting_index=0
            )
        )
        
        print(f"Propagation predictions:")
        for output_name, values in predictions.outputs.items():
            print(f"  {output_name}: {len(values)} predictions")
            print(f"    First 3 values: {values[:3]}")
        
        if predictions.confidence_intervals:
            print(f"\n  Confidence intervals:")
            for var, intervals in predictions.confidence_intervals.items():
                print(f"    {var}: {list(intervals.keys())}")

### 9.3 Cultivation Historical Model Predictions

In [ ]:
from dhl_sdk import HistoricalPredictionInput

# Find a historical model
if cultivation_projects:
    project = cultivation_projects[0]
    hist_models = client.list_models(project, model_type="historical")
    
    if hist_models and hist_models[0].is_ready:
        model = hist_models[0]
        
        # Make historical predictions
        timestamps = [0, 24, 48, 72, 96, 120, 144, 168]  # Hours
        steps = [0, 1, 2, 3, 4, 5, 6, 7]  # Step numbers
        
        predictions = model.predict(
            client,
            HistoricalPredictionInput(
                timestamps=timestamps,
                steps=steps,
                unit="h",  # Hours
                inputs={
                    # X variables: values for each timestamp (can have gaps)
                    "TEMP": [37.0, 37.1, 37.0, 37.1, 37.0, 37.1, 37.0, 37.1],
                    "GLC": [5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5],
                    
                    # W variables and flows: values for each timestamp
                    "FEED": [0.0, 0.1, 0.1, 0.2, 0.2, 0.3, 0.3, 0.4],
                    
                    # Z variables and feed concentrations: initial value only
                    "PHASE": ["Growth"],
                },
                confidence=0.8
            )
        )
        
        print(f"Historical predictions:")
        for output_name, values in predictions.outputs.items():
            print(f"  {output_name}: {len(values)} predictions")
            print(f"    First 3 values: {values[:3]}")

### 9.4 Prediction with Different Time Units

In [ ]:
# You can use different time units: 's' (seconds), 'm' (minutes), 'h' (hours), 'd' (days)

# Example with days
if prop_models and prop_models[0].is_ready:
    model = prop_models[0]
    
    predictions_days = model.predict(
        client,
        PropagationPredictionInput(
            timestamps=[0, 1, 2, 3, 4, 5, 6, 7],  # Days
            unit="d",  # Days
            inputs={
                "TEMP": [37.0],
                "GLC": [5.0],
            },
            confidence=0.9  # 90% confidence
        )
    )
    
    print("Predictions with daily timestamps")
    for output_name, values in predictions_days.outputs.items():
        print(f"  {output_name}: {len(values)} predictions")

## 10. Error Handling

The SDK provides specific error types for different failure scenarios.

### 10.1 Validation Errors

In [ ]:
from dhl_sdk import ValidationError

# Example: Invalid product code (empty)
try:
    invalid_product = client.create_product(
        code="",  # Empty code
        name="Invalid Product"
    )
except ValidationError as e:
    print(f"Validation Error: {e.message}")
    print(f"  Field: {e.field}")
    print(f"  Value: {e.value}")

# Example: Invalid variable group
try:
    invalid_var = client.create_variable(
        code="TEST",
        name="Test Variable",
        unit="unit",
        group="Invalid Group",  # Non-existent group
        type=NumericVariable()
    )
except ValidationError as e:
    print(f"\nValidation Error: {e.message}")
    print(f"  Field: {e.field}")

### 10.2 Entity Already Exists Errors

In [ ]:
from dhl_sdk import EntityAlreadyExistsError

# Try to create duplicate product
try:
    duplicate = client.create_product(
        code="CHO001",  # Same code as before
        name="Duplicate Product"
    )
except EntityAlreadyExistsError as e:
    print(f"Entity Already Exists: {e.message}")
    print(f"  Entity Type: {e.entity_type}")
    print(f"  Identifier: {e.entity_identifier}")

### 10.3 Entity Not Found Errors

In [ ]:
from dhl_sdk import EntityNotFoundError

# Try to get non-existent product
try:
    missing = client.get_product("non-existent-id")
except EntityNotFoundError as e:
    print(f"Entity Not Found: {e.message}")
    print(f"  Entity Type: {e.entity_type}")
    print(f"  Entity ID: {e.entity_id}")

### 10.4 Authentication Errors

In [ ]:
from dhl_sdk import AuthenticationError

# Invalid API key
try:
    bad_client = DataHowLabClient(
        api_key="invalid-key",
        base_url="https://yourdomain.datahowlab.ch/"
    )
    products = bad_client.list_products()
except AuthenticationError as e:
    print(f"Authentication Error: {e.message}")

### 10.5 API Errors

In [ ]:
from dhl_sdk import APIError, ServerError, RateLimitError

# Generic API error handling
try:
    # Some operation that might fail
    products = client.list_products()
except ServerError as e:
    print(f"Server Error: {e.message}")
    print(f"  Status Code: {e.status_code}")
except RateLimitError as e:
    print(f"Rate Limit Error: {e.message}")
    print(f"  Retry After: {e.retry_after} seconds")
except APIError as e:
    print(f"API Error: {e.message}")
    if e.status_code:
        print(f"  Status Code: {e.status_code}")

### 10.6 Prediction Errors

In [ ]:
from dhl_sdk import PredictionError

# Try to predict with model not ready
if models:
    # Create a model object with non-ready status for demo
    from dhl_sdk.models import Model
    
    not_ready_model = Model(
        id="demo-id",
        name="Demo Model",
        status="training",  # Not ready
        project=project.id,
        model_type="propagation"
    )
    
    try:
        predictions = not_ready_model.predict(
            client,
            PropagationPredictionInput(
                timestamps=[0, 1, 2],
                unit="h",
                inputs={"TEMP": [37.0]}
            )
        )
    except ValidationError as e:
        print(f"Model Not Ready: {e.message}")
        print(f"  Field: {e.field}")
        print(f"  Value: {e.value}")

## Summary

This notebook covered all the major functions available in the DataHowLab SDK v2.0:

1. **Client Initialization** - Setting up the SDK client with various options
2. **Product Operations** - Creating, listing, and retrieving products
3. **Variable Operations** - Creating all types of variables (numeric, categorical, logical, flow, spectrum)
4. **Experiment Operations** - Creating experiments with time-series and spectra data, retrieving experiment data
5. **Recipe Operations** - Creating and listing setpoint recipes
6. **Project Operations** - Listing and retrieving projects
7. **Model Operations** - Listing models and accessing model information
8. **Dataset Operations** - Listing datasets within projects
9. **Prediction Operations** - Making predictions with spectroscopy, propagation, and historical models
10. **Error Handling** - Handling various error types gracefully

For more information, visit: https://github.com/DataHow/datahowlab-sdk-python